#### APPLYING ML MODELS TO THE NEXT COMING MATCHES

In [182]:
# Importing the coming matchs with the odds

coming_round = pd.read_excel('coming_round.xlsx', names=['index','id','HomeTeam','AwayTeam','B365H','B365D','B365A'], dtypes = ['int','int','str','str','float','float','float'])
coming_round.drop(['index'], axis = 1, inplace=True)

##### Setting up the coming matches witht the same content and format than the trained dataset

In [183]:
#Current season 
current_season = max(historical_data['season'])

# Filtering historical dataset by current season
historical_acc_current_season = historical_acc[historical_acc['season'] == current_season]

# Calculating the latest match played by each team
historical_acc_current_season_round = historical_acc_current_season.groupby('Team')[['match']].max().reset_index()

# Historical dataset figures at the latest game
historical_acc_current_season_match = historical_acc_current_season_round.merge(historical_acc_current_season, left_on=['Team','match'],right_on=['Team','match'])

# We need to adjust accumulated figures and update classification

historical_acc_current_season_match['goals_for_acc'] = historical_acc_current_season_match['goals_for_acc'] + historical_acc_current_season_match['goals_for']
historical_acc_current_season_match['goals_against_acc'] = historical_acc_current_season_match['goals_against_acc'] + historical_acc_current_season_match['goals_against']
historical_acc_current_season_match['shots_for_acc'] = historical_acc_current_season_match['shots_for_acc'] + historical_acc_current_season_match['shots_for']
historical_acc_current_season_match['shots_against_acc'] = historical_acc_current_season_match['shots_against_acc'] + historical_acc_current_season_match['shots_against']
historical_acc_current_season_match['shots_target_for_acc'] = historical_acc_current_season_match['shots_target_for_acc'] + historical_acc_current_season_match['shots_target_for']
historical_acc_current_season_match['shots_target_against_acc'] = historical_acc_current_season_match['shots_target_against_acc'] + historical_acc_current_season_match['shots_target_against']
historical_acc_current_season_match['points_acc'] = historical_acc_current_season_match['points_acc'] + historical_acc_current_season_match['points']
historical_acc_current_season_match['ranking'] = historical_acc_current_season_match['points_acc'].rank(method='first', ascending=False)

# Merging coming round and accumulated figures

coming_round_home = coming_round.merge(historical_acc_current_season_match[['Div','season','match','Team','goals_for_acc','goals_against_acc','shots_for_acc','shots_against_acc','shots_target_for_acc','shots_target_against_acc','points_acc','ranking']], left_on='HomeTeam', right_on ='Team')
coming_round_away = coming_round.merge(historical_acc_current_season_match[['Div','season','match','Team','goals_for_acc','goals_against_acc','shots_for_acc','shots_against_acc','shots_target_for_acc','shots_target_against_acc','points_acc','ranking']], left_on='AwayTeam', right_on ='Team', suffixes=('','_away'))
coming_round_aux = coming_round_home[['Div','season','match','HomeTeam','AwayTeam','goals_for_acc','goals_against_acc','shots_for_acc','shots_against_acc','shots_target_for_acc','shots_target_against_acc','points_acc','ranking','B365H','B365D','B365A']]

# Adjusting the accumulated numbers

coming_round_aux['HomeCode'] = lab_enc_teams.fit_transform(coming_round_aux[["HomeTeam"]].values.reshape(-1,))
coming_round_aux['AwayCode'] = lab_enc_teams.fit_transform(coming_round_aux[["AwayTeam"]].values.reshape(-1,))
coming_round_aux['goals_for_acc'] = coming_round_aux.loc[:,'goals_for_acc'] - coming_round_away.loc[:,'goals_for_acc']
coming_round_aux['goals_against_acc'] = coming_round_aux['goals_against_acc'] - coming_round_away['goals_for_acc']
coming_round_aux['shots_for_acc'] = coming_round_aux['shots_for_acc'] - coming_round_away['shots_for_acc']
coming_round_aux['shots_against_acc'] = coming_round_aux['shots_against_acc'] - coming_round_away['shots_against_acc']
coming_round_aux['shots_target_for_acc'] = coming_round_aux['shots_target_for_acc'] - coming_round_away['shots_target_for_acc']
coming_round_aux['shots_target_against_acc'] = coming_round_aux['shots_target_against_acc'] - coming_round_away['shots_target_against_acc']
coming_round_aux['points_acc'] = coming_round_aux['points_acc'] - coming_round_away['points_acc']
coming_round_aux['ranking'] = coming_round_aux['ranking'] - coming_round_away['ranking']
coming_round_aux.drop(['Div','HomeTeam','AwayTeam'], axis = 1, inplace = True)

# Changing season & match types

coming_round_aux['season'] = coming_round_aux['season'].astype(int)
coming_round_aux['match'] = (coming_round_aux['match'] +1).astype(int)

# Predicting clusters to the coming matches
                 
coming_round_scaled = pd.DataFrame(scaler.transform(coming_round_aux.iloc[:,0:10]), columns = coming_round_aux.columns[0:10])
coming_round_cluster_predictions = kmeans.predict(coming_round_scaled)

# Final coming round

coming_round_final = coming_round_aux.iloc[:,np.r_[0:10,13,14]]

##### Applying the prediction results from the different ML models 

In [184]:
coming_round_all = pd.DataFrame()

for model in models:
    
    # Calculating predictions and probabilities to the coming matches
    
    coming_round_final = coming_round_aux.iloc[:,np.r_[0:10,13,14]]             
    predictions_coming_round_aux = models[model][2].predict(coming_round_final)
    predictions_coming_round = lab_enc_results.inverse_transform(predictions_coming_round_aux)
    predictions_prob_coming_round = models[model][2].predict_proba(coming_round_final)


    # Adding clusters and probs to the coming matches
                 
    coming_round_final['cluster'] = coming_round_cluster_predictions
    coming_round_final['prediction'] = predictions_coming_round
    coming_round_final['Prob_Home%'] = np.round(predictions_prob_coming_round[:,2]*100,2)
    coming_round_final['Prob_Draw%'] = np.round(predictions_prob_coming_round[:,1]*100,2)
    coming_round_final['Prob_Away%'] = np.round(predictions_prob_coming_round[:,0]*100,2) 
    coming_round_final['model'] = model
    
    # Adding the quotas and teams
    
    bets = coming_round_aux[['B365H','B365D','B365A']]
    teams = coming_round[['HomeTeam','AwayTeam']]
    coming_round_final = pd.concat([coming_round_final,bets, teams], axis = 1)
    
    # Aggregating data
    
    coming_round_all  = coming_round_all.append(coming_round_final)

In [185]:
# Exporting the results

coming_round_all.to_excel('user_report/coming_round_all.xlsx')